In [6]:
# !pip install protobuf
# !pip install sentence-transformers
# !pip install transformers datasets evaluate rouge_score


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import sys 
sys.path.append("./model_grpc/")
from model_server_pb2 import ( modelRequest , modelResponse , modelName , modelInfo , modelConfig)
# from model_server_pb2_grpc import *
import grpc

# connect server
channel = grpc.insecure_channel("140.127.208.185:50051")
client = sendToModelStub(channel)



'''
# 問問題
user = 2
request = modelRequest(user = user , modelName = "vicuna", prompt = "Tell me more")
rel = client.getModelResponse(request)

# 檢查狀態
rel = client.checkModelState(modelName(modelName = "vicuna"))

# 創建model
rel = client.createModelProc(modelConfig(modelName = "vicuna" , maxToken = 50))

# 刪除model
rel = client.deleteModelProc(modelName(modelName = "vicuna"))

'''


prompt: "Tell me more"
response: "HUMAN: Tell me more\nASSISTANT: The Human is a science fiction novel by author and futurist James P. Nestsus. The book was published in 1981 and tells the story of a man named John who is cryogenically frozen in the year 1981 and awakens in the year 2081.\n\nThe story is set in a future where humanity has colonized other planets and has developed advanced technology, including the ability to cryogenically freeze and revive people. The protagonist, John, is a scientist who is cryogenically frozen in order to preserve his knowledge and skills for the future.\n\nWhen John is revived in the year 2081,"



'\n# 問問題\n\n\n# 檢查狀態\nrel = client.checkModelState(modelName(modelName = "vicuna"))\n\n# 創建model\nrel = client.createModelProc(modelConfig(modelName = "vicuna" , maxToken = 50))\n\n# 刪除model\nrel = client.deleteModelProc(modelName(modelName = "vicuna"))\n\n'

In [23]:
import pandas as pd 
import bisect

#### Read story
#### Get high light and save it
* high light 傳來會是一個有start idx 和 end idx的


In [48]:
# find whether has highlight in this paragraph  
def find_has_highlight(start_idx:int,end_idx:int,keys:list)->list:
    
    # save key of highlight
    list_high_light = list()
    # find start highlight ,lower_bound
    idx= bisect.bisect_left(keys,start_idx)
    
    try:
        # find others until greater than end_idx
        for i in range(idx,len(keys)):
            if keys[i] <= end_idx:
                list_high_light.append(keys[i])
    except:
        # there isn't highlight in this paragraph
        pass

    return list_high_light

In [25]:
# Load model
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification ,pipeline ,BartForConditionalGeneration

model_paragraph_similarity = SentenceTransformer('paraphrase-MiniLM-L6-v2')

tokenizer_get_paragraph_name = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model_get_paragraph_name = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")


tokenizer_summarize = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model_summarize = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


In [39]:

def paragraph_similarity(model,paragraph1:str,paragraph2:str):
    

    sentences = [paragraph1,paragraph2]

    # Compute the embeddings
    embeddings = model.encode(sentences)

    # Compute the cosine similarity between the embeddings
    from sklearn.metrics.pairwise import cosine_similarity
    similarity = cosine_similarity(embeddings)
    print(similarity[0][1])
    return similarity[0][1]
    

    

In [41]:

def summarize(paragraph:str):
    inputs = tokenizer_summarize(paragraph, max_length=1024, return_tensors="pt",truncation=True)
    # Generate Summary
    summary_ids = model_summarize.generate(inputs["input_ids"], num_beams=3, min_length=20, max_length=450)
    summarize_content = tokenizer_summarize.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    return summarize_content



In [28]:

def get_paragraph_name(model,paragraph:str,threshold)->list():
    

    nlp = pipeline("ner", model=model, tokenizer=tokenizer_get_paragraph_name)
    ner_results = nlp(paragraph)

    
    table = ["B-PER","I-PER"]
    names = list()
    
    for ner in ner_results:
        if ner["entity"] in table and ner["score"] >= threshold:
            
            if ner["word"] not in names:
                names.append(ner["word"])
    
    if len(names) == 0 : return None
    
    return names


In [29]:
# Read story
story_file_path = "./story/content/using.txt"
with open(story_file_path,"r") as file :
    # remove space and "\n" of head and tail of paragraph 
    content = file.read().strip()
file.close()


# Read highlight
highlight_file_path = "./story/highlight/using.txt"
with open(highlight_file_path,"r") as file:
    
    dict_highlight=dict()
    line = file.readline()
    while line :
        start_idx , end_idx = line.strip('\n').split(" ")
        line = file.readline()
        dict_highlight[int(start_idx)] = int(end_idx)

# Sorted order of dict's key 
copy = dict_highlight.copy()
keys = sorted(dict_highlight.keys())
dict_highlight.clear()
for key in keys:
    dict_highlight[key] = copy[key]
    
print(dict_highlight)

# Testing highlight 
for key in dict_highlight.keys():
    str = content[key:dict_highlight[key]+1]
    print(str)





{91: 275, 506: 662, 1427: 1575, 2157: 2304, 2333: 2589}
The kids at school teased me and were cruel to me. I was 14 when the most popular girl at school, Jessica, poured juice on my head. Everyone laughed as I cried in the bathroom, ashamed.
Mary put suspicious pills in Jessica's backpack the next day and called the police. Jessica was in shock when two police officers found the pills in her bag.
I heard the news that the boy who kissed me at the party was in the hospital. Supposedly, someone pushed him as he went down the steps to the subway.
The school bell rang and suddenly all the kids in the school were standing in the hallway laughing at me. I felt anger, standing naked and ashamed. 
She turned into a devil when I told her what had happened. I begged her not to do anything, but Mary didn't listen to me. She opened her backpack, took out her books, and lit them with a match. Within a minute, the fire started spreading through the school.


In [30]:
# Parameter 

# Each input size
INPUT_SIZE = 512
# Whether similarity
THRESHOLD_SIMILARITY = 0.6
# Whether similarity
THRESHOLD_IS_NAME = 0.9

In [50]:

current_idx = 0
# 上一個子段落
last_sub_paragraph =""
while current_idx < len(content):
    
    # Find complete paragraph
    end_idx = min(current_idx + INPUT_SIZE-1,len(content)-1)
    end_idx = content.rfind('.',current_idx,end_idx+1)
    
    # Do processing 

    # get sub_paragraph
    sub_paragraph = content[current_idx:end_idx+1]
    # Highlight in the paragraph
    list_has_highlight = find_has_highlight(start_idx=current_idx,end_idx=end_idx,keys=keys)
    
    this_paragraph_highlight = list()
    has_highlight =True
    try:
        for i in list_has_highlight:
            highlight_content = content[i:dict_highlight[i]+1]
            this_paragraph_highlight.append(highlight_content)
    except:
        has_highlight = False
    
   
    # 重點擷取
    # 要換成重點擷取後
    summarize_paragraph = summarize(sub_paragraph)
    # 檢查重點中是否有包含highlight內容
    if has_highlight:
        for highlight in this_paragraph_highlight:
            
            # print(highlight)
            # print(sub_paragraph)
            # print(summarize_paragraph)
            # print("!")
            
            # 大於特定值就當作已經有提到，把highlight中刪除 
            if paragraph_similarity(model = model_paragraph_similarity,paragraph1 = highlight,paragraph2 = summarize_paragraph) >= THRESHOLD_SIMILARITY:
                this_paragraph_highlight.remove(highlight)

    # 檢查是否有人名
    paragraph_name = get_paragraph_name(model = model_get_paragraph_name,paragraph=summarize_paragraph,threshold=THRESHOLD_IS_NAME)
    print(summarize_paragraph)
    print(this_paragraph_highlight)
    print(paragraph_name)
    
    # 確認是否有生成過
    # 沒有就找特徵，QA問題，或是generation
    # 臉部特徵+ 重點擷取直接畫
    # 


    # next paragraph
    last_sub_paragraph = sub_paragraph
    current_idx = end_idx+1


0.56692034
Tina was 14 when the most popular girl at school, Jessica, poured juice on her head. When she returned to the classroom, she saw a new student, Mary, who told her she should take revenge.
['The kids at school teased me and were cruel to me. I was 14 when the most popular girl at school, Jessica, poured juice on my head. Everyone laughed as I cried in the bathroom, ashamed.']
['Tina', 'Jessica', 'Mary']
1.0
Mary put suspicious pills in Jessica's backpack the next day and called the police. Jessica was in shock when two police officers found the pills in her bag.
[]
['Mary', 'Jessica']
When the boy asked if I wanted to kiss him, I blushed and ran away. I didn't think much of it, though, so I went to the couch, leaned back on it, and fell asleep. When I woke up in the morning, the boy approached me and kissed me. I was shocked. I decided to tell Mary about this. She was pissed, telling me not to worry and it wasn't my fault.
[]
['Mary']
0.78247094
The boy who kissed me at the p